# Data Cleaning in SQL


## Point 1 : 

**Objective** : Convert column 'saledate' datatype to date data type.

**Solution** : Possible with code but I decided to use Dbeaver (my db management tool) in the property settings. (Easier and faster)

## Point 2 

**Objective** : Populate missing data in the column propertyaddress:

**Solution** : Use of SELF JOIN + COALESCE + UPDATE TABLE

In [ ]:
select 
	a.parcelid,
	a.propertyaddress, 
	b.parcelid , 
	b.propertyaddress,
	coalesce (a.propertyaddress,b.propertyaddress) as newcolumn
from housing a
join housing b 
	on a.parcelid = b.parcelid 
	and a.id <> b.id
where a.propertyaddress is null

UPDATE housing a
SET propertyaddress = coalesce(a.propertyaddress, b.propertyaddress)
FROM housing b
WHERE a.propertyaddress IS NULL
AND a.parcelid = b.parcelid;

## Point 3 

**Objective** : Break out propertyaddress column into individual columns 

**Solution**:

First figure out how to extract address and city from propertyaddress column

In [ ]:
select 	
	substring(propertyaddress, 1,position(',' in propertyaddress) -1) as address,
	substring(propertyaddress, position(',' in propertyaddress) + 1, length(propertyaddress)) as town
from housing h 

*And then simply add two columns with the adequate copy pasted code above*

In [ ]:
alter table housing 
add PropertySplitAddress varchar(255);

update housing 
set PropertySplitAddress = substring(propertyaddress, 1,position(',' in propertyaddress) -1)

alter table housing 
add PropertySplitCity varchar(255);

update housing 
set PropertySplitCity = substring(propertyaddress, position(',' in propertyaddress) + 1, length(propertyaddress))

*There’s a fairly easier way to split columns in Postgresql.*
*Let’s say we want to separate address, city and state on owner address. The following code would work:*

In [ ]:
SELECT 
    split_part(owneraddress, ',', 1) as address,
    split_part(owneraddress, ',', 2) as city,
    split_part(owneraddress, ',', 3) as state
FROM housing;

*And then we can just add the 3 columns :*

In [ ]:
alter table housing 
add OwnerSplitAddress varchar(255);

update housing 
set ownersplitaddress = split_part(owneraddress, ',', 1);

*That does it for the first added column ….*


## Point 4

**Objective** : Clean up column soldasvacant inconsistencies.

Sometimes we have “y” for “yes” sometimes “N” for “No”

**Solution:**

In [ ]:
select
	distinct(soldasvacant),
	count(soldasvacant)
from housing
group by distinct(soldasvacant);


We can now create the select statement …

select 
	soldasvacant, 
	case when soldasvacant = 'N' then 'No'
		 when soldasvacant  = 'Y' then 'Yes'
	else soldasvacant 
	end as newcolumn
from housing;
	
And correct the column :

update housing
set soldasvacant = 
	case when soldasvacant = 'N' then 'No'
		 when soldasvacant  = 'Y' then 'Yes'
			else soldasvacant 
			end


## Point 5 

**Objective** : remove duplicates

**Solution** : Excel! Much easier than in SQL with CTE statements.

## POINT 6

**Objective** : DROP columns


In [ ]:
Alter table housing
drop column owneraddress, 
drop column taxdistrict;